In [5]:
import sys
sys.path.append("../..")
from copy import deepcopy
import dvc.api
import pandas as pd
import torch
import numpy as np
from main_multitask_multimodal import LLM_MultitaskMultimodal
import yaml
md = {'clm_multimodal_clip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]}},
 'dlm_multimodal_wishtitlewclip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]}},
 'seqclf_multimodal_wishtitlewclip2pseudov121tax': {'multimodal_embedding': {'input': [{'key': 'img_embedding',
     'proj_head': 'proj_head'}]},
  'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}},
 'emb_singlemodal_wishquery2googletitle': None,
 'clm_singlemodal_alititle2v121tax': None,
 'clm_singlemodal_wishtitle2pseudov121tax': None,
 'dlm_singlemodal_wishtitle': None,
 'emb_singlemodal_wishtitle2pseudov121tax': None,
 'emb_singlemodal_alititle2v121tax': None,
 'seqclf_singlemodal_alititle2v121tax': {'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}},
 'seqclf_singlemodal_wishtitle2pseudov121tax': {'specs': {'clf_head': 'clf_head',
   'clf_weight_type': None,
   'label_map_file': '../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt',
   'label_type': 'taxonomy'}}}
model = LLM_MultitaskMultimodal.load_from_checkpoint(
    '../../models/product_title_multitask_multimodal/version_1/epoch=0-step=75000.ckpt',
    multitask_specs_dict = md
)

INFO:root:Unused kwargs when getting t5-base: {'distance_func': 'cosine', 'loss_type': 'cross-entropy', 'margin': None, 'hidden_states_type': 'encoder-last', 'add_simcse': False, 'manual_loss_type': 'manual_mse', 'auto_task_weight': False, 'multitask_specs_dict': {'clm_multimodal_clip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}}, 'dlm_multimodal_wishtitlewclip2wishtitle': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}}, 'seqclf_multimodal_wishtitlewclip2pseudov121tax': {'multimodal_embedding': {'input': [{'key': 'img_embedding', 'proj_head': 'proj_head'}]}, 'specs': {'clf_head': 'clf_head', 'clf_weight_type': None, 'label_map_file': '../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'label_type': 'taxonomy'}}, 'emb_singlemodal_wishquery2googletitle': None, 'clm_singlemodal_alititle2v121tax': None, 'clm_singlemodal_wishtitle2pseudov121tax': None, 'dlm_singlemodal_wishtitle': None, 

In [26]:
model.eval()
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    LogitsProcessorList,
    MinLengthLogitsProcessor,
    HammingDiversityLogitsProcessor,
    BeamSearchScorer,
)
import torch


In [53]:
from main_utils import Trie

In [54]:
allowed_gen_sequences = []
with open('../../datasets/taxonomy/wish_v1.2.1_newtax_allpaths.txt', 'r') as f:
    for l in f:
        if len(l):
            allowed_gen_sequences.append(l)

In [55]:
trie = Trie([
    [model.tokenizer.pad_token_id] + model.tokenizer.encode(i) + \
        [model.tokenizer.eos_token_id] for i in allowed_gen_sequences
])
def constraint(batch_id, sent):
    return trie.get(sent.tolist())

In [102]:
df_eval = pd.read_excel('../eval_query/tmp_test_eval.xlsx')

In [120]:
df_eval[(df_eval.y_pred_jc == 0) & (df_eval.y_baseline_jc > 0)][[
    'query', 'y_true', 'y_pred', 'y_baseline']].sample(1).to_dict('records')

[{'query': 'tasers',
  'y_true': "['security & protection > self defense supplies']",
  'y_pred': "['sports > roller skates, skateboards & scooters > skate shoes', 'education & office supplies > books & magazines > humanities & social science', 'sports > fitness & body building > martial clothing > martial pants']",
  'y_baseline': "['security & protection > self defense supplies', 'security & protection', 'home improvement > lights & lighting > portable lighting > flashlights & torches']"}]

In [121]:
inputs = model.tokenizer("Generate taxonomy: tasers", return_tensors="pt")

In [122]:
outs = model.transformer.generate(inputs['input_ids'], num_beams=3, prefix_allowed_tokens_fn=constraint,
    num_return_sequences=3, output_scores=True, return_dict_in_generate=True, length_penalty=1)
model.tokenizer.batch_decode(outs.sequences), outs.sequences_scores.exp()

/opt/conda/envs/py38/lib/python3.8/site-packages/transformers/generation_utils.py:1227: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


(['<pad> sports > fitness & body building > martial clothing > martial pants</s><pad><pad>',
  '<pad> home & garden > home textile > bedding > mattress toppers</s><pad><pad>',
  '<pad> sports > fitness & body building > boxing > boxing trunks</s>'],
 tensor([0.7990, 0.7255, 0.7092]))

In [123]:
outs = model.transformer.generate(inputs['input_ids'], num_beams=3, prefix_allowed_tokens_fn=constraint,
    num_return_sequences=3, output_scores=True, return_dict_in_generate=True, length_penalty=0)
model.tokenizer.batch_decode(outs.sequences), outs.sequences_scores.exp()

(['<pad> sports > fitness & body building > martial clothing > martial pants</s><pad><pad>',
  '<pad> home & garden > home textile > bedding > mattress toppers</s><pad><pad>',
  '<pad> sports > fitness & body building > boxing > boxing trunks</s>'],
 tensor([0.0432, 0.0112, 0.0041]))